In [ ]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator_OnlyVel as Generator 
import model_definitions

### Open input file

In [9]:
SEQUENCE_LENGTH=30
ONE_CAR=True
PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_Pos_Vel_Rot.txt"
BATCH_SIZE=256*5
FEATURES=5 
FEATURES_PREDICTED=3

In [10]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [11]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
df=Generator.single_care_dataframe(PATH)

In [12]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [13]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [14]:
df_train.head()

,X,Z,DIFF_VEL_X,DIFF_VEL_Z,DIFF_ROT,TIME,RACE,GROUP
0,0.00000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.00,0,0
1,-64.90002,3.429016,0.000000e+00,0.959571,0.000000e+00,0.06,0,0
2,-64.90002,3.538513,0.000000e+00,1.277536,0.000000e+00,0.06,0,0
3,-64.90002,3.722473,-3.956728e-11,1.284701,-1.505327e-09,0.06,0,0
4,-64.90002,3.982544,-5.000784e-10,1.224364,-5.415302e-09,0.06,0,0


In [15]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 7294690 races: 448 n batches: 7240930 / 1280
Length: 3647349 races: 448 n batches: 3593589 / 1280
Length: 3648017 races: 448 n batches: 3594257 / 1280


### Model definition

#### Train and print

In [16]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [12]:
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=10,verbose=1,name="MLP",path="logs_deltaFixedUpdate_Iloc")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/MLP0001_PosAssoluta')

Epoch 1/10
5655/5655 [==============================] - 1957s 346ms/step - loss: 158.5340 - mean_absolute_error: 2.9998 - accuracy: 0.0000e+00 - val_loss: 0.3061 - val_mean_absolute_error: 0.3822 - val_accuracy: 9.2807e-08 - lr: 1.0000e-04
Epoch 2/10
5655/5655 [==============================] - 1947s 344ms/step - loss: 0.2575 - mean_absolute_error: 0.3322 - accuracy: 0.0000e+00 - val_loss: 0.2204 - val_mean_absolute_error: 0.2960 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5655/5655 [==============================] - 1963s 347ms/step - loss: 0.2581 - mean_absolute_error: 0.3241 - accuracy: 4.6051e-08 - val_loss: 0.2206 - val_mean_absolute_error: 0.2969 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5655/5655 [==============================] - ETA: 0s - loss: 0.2531 - mean_absolute_error: 0.3232 - accuracy: 4.6051e-08
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
5655/5655 [==============================] - 1946s 344ms/step - loss: 0.2531

In [13]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

Epoch 1/10
5655/5655 [==============================] - 2340s 413ms/step - loss: 0.9373 - mean_absolute_error: 0.6307 - accuracy: 4.6051e-08 - val_loss: 0.7536 - val_mean_absolute_error: 0.5764 - val_accuracy: 9.2807e-08 - lr: 1.0000e-04
Epoch 2/10
5655/5655 [==============================] - 2301s 407ms/step - loss: 0.5683 - mean_absolute_error: 0.5047 - accuracy: 0.0000e+00 - val_loss: 0.4222 - val_mean_absolute_error: 0.4416 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5655/5655 [==============================] - 2333s 412ms/step - loss: 0.3508 - mean_absolute_error: 0.4066 - accuracy: 0.0000e+00 - val_loss: 0.2898 - val_mean_absolute_error: 0.3652 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5655/5655 [==============================] - 2304s 407ms/step - loss: 0.2785 - mean_absolute_error: 0.3564 - accuracy: 0.0000e+00 - val_loss: 0.2501 - val_mean_absolute_error: 0.3341 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5655/5655 [==========================

INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


In [26]:
lr =1e-5
units=32
cells=2
normalization=False
name=f"LSTM{cells}_{units}_{lr}_Norm{normalization}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs_NoPos",
                                                     epoch=5, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

Epoch 1/5
5655/5655 [==============================] - 2665s 470ms/step - loss: 0.8589 - mean_absolute_error: 0.6214 - accuracy: 0.0000e+00 - val_loss: 0.9392 - val_mean_absolute_error: 0.6785 - val_accuracy: 2.7842e-07 - lr: 1.0000e-05
Epoch 2/5
5655/5655 [==============================] - 2777s 491ms/step - loss: 0.6434 - mean_absolute_error: 0.5314 - accuracy: 9.2101e-08 - val_loss: 0.8209 - val_mean_absolute_error: 0.6206 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 3/5
5655/5655 [==============================] - 2647s 468ms/step - loss: 0.6034 - mean_absolute_error: 0.5082 - accuracy: 4.6051e-08 - val_loss: 0.7608 - val_mean_absolute_error: 0.5913 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 4/5
5655/5655 [==============================] - 2679s 474ms/step - loss: 0.5788 - mean_absolute_error: 0.4949 - accuracy: 1.3815e-07 - val_loss: 0.7231 - val_mean_absolute_error: 0.5725 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 5/5
5655/5655 [==============================]

In [27]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}_PosAssoluta')

INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
y_p=model.predict(x_ten)

y_p

In [ ]:
y=x.iloc[SEQUENCE_LENGTH+1]
diff= y.values-y_p
print(y)
diff